In [38]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

class AssignmentProblem:
    def __init__(self, filename):
        self.filename = filename
        self.num_tasks = None
        self.cost_matrix = None
        self.model = None
        self.assignments = []

    def read_data(self):
        """Reads the data from the file and parses the cost matrix."""
        print("Reading data from the file...")
        with open(self.filename, 'r') as file:
            data_lines = [line.strip() for line in file if line.strip() and not line.startswith('#')]

        self.num_tasks = int(data_lines[0])
        flat_cost_data = [int(x) for line in data_lines[1:] if "EOF" not in line for x in line.split()]
        assert len(flat_cost_data) == self.num_tasks ** 2, f"Expected {self.num_tasks ** 2} entries, got {len(flat_cost_data)}"

        self.cost_matrix = np.array([flat_cost_data[i * self.num_tasks:(i + 1) * self.num_tasks] for i in range(self.num_tasks)])
        print(f"Successfully parsed a {self.num_tasks}x{self.num_tasks} cost matrix!")

    def build_model(self):
        """Builds the Gurobi optimization model."""
        print("Building the optimization model...")
        self.model = gp.Model("Task Assignment")

        # Decision variables
        self.x = self.model.addVars(self.num_tasks, self.num_tasks, vtype=GRB.BINARY, name="x")

        # Objective: Minimize total assignment cost
        self.model.setObjective(
            gp.quicksum(self.cost_matrix[i][j] * self.x[i, j] for i in range(self.num_tasks) for j in range(self.num_tasks)),
            GRB.MINIMIZE
        )

        # Constraints
        print("Adding constraints...")
        # Each task must be assigned to exactly one employee
        for i in range(self.num_tasks):
            self.model.addConstr(gp.quicksum(self.x[i, j] for j in range(self.num_tasks)) == 1, name=f"Task_{i}")

        # Each employee must be assigned to exactly one task
        for j in range(self.num_tasks):
            self.model.addConstr(gp.quicksum(self.x[i, j] for i in range(self.num_tasks)) == 1, name=f"Employee_{j}")

        print("Model building complete.")

    def solve(self):
        """Solves the optimization model."""
        if self.model is None:
            raise ValueError("Model has not been built. Call build_model() first.")

        print("Optimizing the model...")
        self.model.optimize()

        if self.model.status == GRB.OPTIMAL:
            print("Optimal solution found!")
            print(f"Total cost: {self.model.objVal}")
        else:
            print("No optimal solution found.")

    def get_assignment(self):
        """Extracts the optimal task-to-employee assignments."""
        if self.model.status == GRB.OPTIMAL:
            print("Extracting assignments...")
            self.assignments = []
            for i in range(self.num_tasks):
                for j in range(self.num_tasks):
                    if self.x[i, j].x > 0.5: 
                        self.assignments.append((i + 1, j + 1))  #1-based indexing
                        print(f"Task {i + 1} assigned to Employee {j + 1}")
        else:
            print("No optimal solution to extract assignments.")

# Example usage
if __name__ == "__main__":
    print("Starting the assignment problem...")
    assignment_problem = AssignmentProblem("instance.txt")
    assignment_problem.read_data()
    assignment_problem.build_model()
    assignment_problem.solve()
    print('-'*100)
    assignment_problem.get_assignment()
    print("Process completed.")


Starting the assignment problem...
Reading data from the file...
Successfully parsed a 100x100 cost matrix!
Building the optimization model...
Adding constraints...
Model building complete.
Optimizing the model...
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 200 rows, 10000 columns and 20000 nonzeros
Model fingerprint: 0x0e0ae492
Variable types: 0 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5140.0000000
Presolve time: 0.02s
Presolved: 200 rows, 10000 columns, 20000 nonzeros
Variable types: 0 continuous, 10000 integer (10000 binary)

Root relaxation: objective 3.050000e+02, 218